# Before running the pipeline, change:
## `Runtime` -->  `Change runtime type` --> `GPU`

In [ ]:
#@title Links that are subject to updates

WEIGHTS = "https://d.aibird.me/8fdadb26.pt"
CLASSES = "https://d.aibird.me/83b3976c.txt"
VIEW_ID = 41
TODAY_VIEW_ID = 42

In [ ]:
%%shell
sudo apt -qq update
sudo apt -qq install ffmpeg libsm6 libxext6  -y
curl https://getcroc.schollz.com | bash

In [ ]:
%%shell
wget -q https://downloads.rclone.org/v1.58.0/rclone-v1.58.0-linux-amd64.zip
unzip rclone-v1.58.0-linux-amd64.zip
mv rclone-v1.58.0-linux-amd64/rclone .

## Run this command on your personal computer:


```shell
croc send /Users/bdschol10/.config/rclone/rclone.conf
```

#### Copy the passphrase part and paste it in the cell input below.


In [ ]:
PASSPHRASE = '' #@param {type:"string"}
if not PASSPHRASE:
    raise Exception('Paste the passphrase in the input bar!')
if 'croc' in PASSPHRASE:
    PASSPHRASE = PASSPHRASE.split('croc ')[1]

! croc --yes $PASSPHRASE

In [ ]:
# this step will take a while, it's normal. Check the logs `out.log` if it takes too long.

! mkdir "downloaded_$(date +%m-%d-%Y)"

! ./rclone --drive-shared-with-me copy gdrive:picam \
  "downloaded_$(date +%m-%d-%Y)" --config "rclone.conf" \
  --transfers 32 -P --stats-one-line >> out.log

# <---- The progress is logged to out.log

! ./rclone size "downloaded_$(date +%m-%d-%Y)"

In [ ]:
%%shell
pip -q install lxml "PyYAML>=5.3.1"

In [ ]:
! git clone https://github.com/ultralytics/yolov5.git
! pip -q install -r yolov5/requirements.txt

! wget "$WEIGHTS" -qO "weights.pt"

In [ ]:
! python yolov5/detect.py --source "downloaded_$(date +%m-%d-%Y)" \
--weights "weights.pt" --save-txt --save-conf

In [ ]:
# this step will take a while, it's normal. Check the logs `move.log` if it takes too long.

! ./rclone --drive-shared-with-me move -P --transfers 32 --stats-one-line \
--max-depth 1 --filter "+ *.jpg" --filter "- *" \
gdrive:"picam" gdrive:"picam-downloaded/downloaded_$(date +%m-%d-%Y)" >> move.log


In [ ]:
from datetime import date
from glob import glob

folder = sorted(glob('yolov5/runs/detect/*'))[-1]
ts = f'downloaded_{date.today().strftime("%m-%d-%Y")}'

In [ ]:
! mkdir $folder/images
! mv $folder/*.jpg $folder/images

! wget "$CLASSES" -qO $folder/classes.txt
! wget "https://d.aibird.me/ab537c0f.py" -qO "yolo.py"

In [ ]:
! cat $folder/classes.txt

In [ ]:
from yolo import convert_yolo_to_ls

try:
    convert_yolo_to_ls(input_dir=folder, out_file='output.json',
        to_name='image', from_name='label', out_type="annotations",
        image_root_url=f'https://ls.aibird.me/data/local-files/?d=picam/{ts}/with_detections',
        image_ext='.jpg')
except AttributeError as e:
    raise AttributeError(f'{e}\n\nFAILED! RESTART THE RUNTIME!')

In [ ]:
from pathlib import Path
_detections = sorted(glob(f'{folder}/labels/*'))

detections = [f'{Path(x).stem}.jpg\n' for x in _detections]

with open('detections.txt', 'w') as f:
    f.writelines(detections)

len(detections)

In [ ]:
! curl -F'file=@detections.txt' https://ttm.sh
print('Copy the link!')

## <---- Click refresh

![refresh](https://d.aibird.me/dc0e384b.png)

## <---- Then, download `output.json`

![download](https://d.aibird.me/02fc8068.png)


In [ ]:
! rm rclone.conf

## On the local server, run the following commands:

```shell
cd "/home/ubuntu/apps/label-studio"

source .env && docker run --rm -v "$PWD/pg_backups:/backups" -e POSTGRES_HOST=${_POSTGRES_HOST} -e POSTGRES_DB=${POSTGRE_NAME} -e POSTGRES_USER=${POSTGRE_USER} -e POSTGRES_PASSWORD=${POSTGRE_PASSWORD} prodrigestivill/postgres-backup-local /backup.sh

rclone copy pg_backups pg_backups: -P --stats-one-line -L
```

```shell
wget <REPLACE_WITH_COPIED_LINK> -O "detections.txt"
```


```shell
NEW_FOLDER_NAME="local-files/picam/downloaded_$(date +%m-%d-%Y)"

rclone --drive-shared-with-me copy gdrive:"picam-downloaded/downloaded_$(date +%m-%d-%Y)" "$NEW_FOLDER_NAME" -P --stats-one-line --transfers 32 --files-from "detections.txt"
```

```shell
mkdir $NEW_FOLDER_NAME/with_detections

mv $NEW_FOLDER_NAME/*.jpg $NEW_FOLDER_NAME/with_detections
```

```shell
python sync_picam.py
```

In [ ]:
! git clone https://github.com/bird-feeder/BirdFSD-YOLOv5.git
%cd BirdFSD-YOLOv5

## Run these commands on the local server:


```shell
cd ~/BirdFSD-YOLOv5
croc send .env
```

#### Copy the passphrase part and paste it in the cell input below.


In [ ]:
PASSPHRASE = '' #@param {type:"string"}
if not PASSPHRASE:
    raise Exception('Paste the passphrase in the input bar!')
if 'croc' in PASSPHRASE:
    PASSPHRASE = PASSPHRASE.split('croc ')[1]

! croc --yes $PASSPHRASE

In [ ]:
! pip -q install -r requirements.txt

In [ ]:
! python update_today_view.py -v $TODAY_VIEW_ID

## In Label-studio:

- Select "Today" view.
- Click the refresh button in label-studio.
- Sort by ID ascending and descnding and get the first and last ID in the filtered tasks view.

In [ ]:
FIRST_ID =  #@param {type: "number"}
LAST_ID =  #@param {type: "number"}

! python apply_predictions_YOLOv5.py -f $FIRST_ID -l $LAST_ID --data "/content/output.json"

In [ ]:
! wget "$CLASSES" -qO classes.txt
! python conditional_tasks_removal.py -v $VIEW_ID -c "classes.txt"

**Follow the instructions in the output above.**

In [ ]:
! rm .env

## Last step:

### Go to `Edit` and click `Clear all ouputs`, then remove the passphrases, first_id, last_id inputs. Then, save.